 # Stacking Ensemble: Meta-Learning Approach

### Score: 0.00000

 This notebook implements **stacking** - training a meta-model to learn optimal combinations of base predictions.



 **Key Innovation**: Instead of fixed weights (0.2, 0.65), a Ridge model learns:

 - Sector-specific combinations

 - Month-specific adjustments

 - How to use historical averages as additional signals

In [47]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
import warnings
warnings.filterwarnings('ignore')


 ## Configuration

In [48]:
# ============================================================
# CONFIGURATION SECTION
# ============================================================

# Data Paths
DATA_PATH = Path("/Users/nikola/Python/KaggleCompetition/data")

# Method 1: Weighted Geometric Mean Configuration
CONFIG_METHOD1 = {
    'n_lags': 6,
    'alpha': 0.5,
    't2': 6,
}

# Method 2: Seasonality Bump Configuration
CONFIG_METHOD2 = {
    'n_lags': 7,
    'alpha': 0.5,
    't2': 6,
    'clip_low': 0.85,
    'clip_high': 1.40,
}

# Stacking Configuration
CONFIG_STACKING = {
    'validation_months': 6,              # Use last 6 months for meta-model training
    'meta_model': 'ridge',               # Options: 'ridge', 'lasso', 'rf'
    'meta_alpha': 1.0,                   # Regularization for Ridge/Lasso
    'lookback_windows': [3, 6, 12],      # Windows for historical averages
    'include_month_feature': True,       # Include month number as feature
    'pre_scale_factor': 0.85,            # Pre-scale predictions before stacking (regime shift)
}

# Output Configuration
CONFIG_OUTPUT = {
    'output_path': '/Users/nikola/Python/KaggleCompetition/output/16_stacking_ensemble',   # Output directory
    'filename': 'stacking_ensemble_submission.csv'         # Submission filename
}

# Display configuration
print("=" * 60)
print("STACKING ENSEMBLE CONFIGURATION")
print("=" * 60)
print("\nMethod 1 - Weighted Geometric Mean:")
for key, value in CONFIG_METHOD1.items():
    print(f"  {key}: {value}")

print("\nMethod 2 - Seasonality Bump:")
for key, value in CONFIG_METHOD2.items():
    print(f"  {key}: {value}")

print("\nStacking Configuration:")
for key, value in CONFIG_STACKING.items():
    print(f"  {key}: {value}")

print("\nOutput Configuration:")
for key, value in CONFIG_OUTPUT.items():
    print(f"  {key}: {value}")
print("=" * 60)


STACKING ENSEMBLE CONFIGURATION

Method 1 - Weighted Geometric Mean:
  n_lags: 6
  alpha: 0.5
  t2: 6

Method 2 - Seasonality Bump:
  n_lags: 7
  alpha: 0.5
  t2: 6
  clip_low: 0.85
  clip_high: 1.4

Stacking Configuration:
  validation_months: 6
  meta_model: ridge
  meta_alpha: 1.0
  lookback_windows: [3, 6, 12]
  include_month_feature: True
  pre_scale_factor: 0.85

Output Configuration:
  output_path: /Users/nikola/Python/KaggleCompetition/output/16_stacking_ensemble
  filename: stacking_ensemble_submission.csv


 ## 1. Load Data

In [49]:
# Load training data
train_nht = pd.read_csv(DATA_PATH / "train" / "new_house_transactions.csv")
test = pd.read_csv(DATA_PATH / "test.csv")

# Convert month to datetime
train_nht['month'] = pd.to_datetime(train_nht['month'])

# Parse test IDs
test_id = test.id.str.split('_', expand=True)
test['month_text'] = test_id[0]
test['sector'] = test_id[1]

# Create month mapping
month_codes = {
    'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
    'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
}

# Add time features to training data
train_nht['year'] = train_nht['month'].dt.year
train_nht['month_num'] = train_nht['month'].dt.month
train_nht['time'] = (train_nht['year'] - 2019) * 12 + train_nht['month_num'] - 1
train_nht['sector_id'] = train_nht.sector.str.slice(7, None).astype(int)

# Parse test data
test['year'] = test['month_text'].str.slice(0, 4).astype(int)
test['month_abbr'] = test['month_text'].str.slice(5, None)
test['month_num'] = test['month_abbr'].map(month_codes)
test['time'] = (test['year'] - 2019) * 12 + test['month_num'] - 1
test['sector_id'] = test.sector.str.slice(7, None).astype(int)

print(f"Training data: {train_nht.shape}")
print(f"Test data: {test.shape}")


Training data: (5433, 15)
Test data: (1152, 9)


 ## 2. Prepare Amount Matrix

In [50]:
# Create pivot table: time x sector_id
amount_matrix = train_nht.set_index(['time', 'sector_id']).amount_new_house_transactions.unstack()
amount_matrix = amount_matrix.fillna(0)

# Add sector 95 (missing in training)
amount_matrix[95] = 0
amount_matrix = amount_matrix[np.arange(1, 97)]

print(f"Amount matrix shape: {amount_matrix.shape}")
print(f"Time range: {amount_matrix.index.min()} to {amount_matrix.index.max()}")


Amount matrix shape: (67, 96)
Time range: 0 to 66


 ## 3. Base Model Prediction Functions

In [51]:
def weighted_geometric_mean_prediction(amount_matrix, n_lags=6, alpha=0.5, t2=6, 
                                      prediction_horizon=12):
    """Method 1: Weighted Geometric Mean with exponential decay"""
    weights = np.array([alpha**(n_lags-1-i) for i in range(n_lags)])
    weights = weights / weights.sum()
    
    # Create prediction dataframe
    last_time = amount_matrix.index.max()
    pred_indices = np.arange(last_time + 1, last_time + 1 + prediction_horizon)
    predictions = pd.DataFrame(index=pred_indices, columns=amount_matrix.columns, dtype=float)
    
    for sector in amount_matrix.columns:
        if (amount_matrix.tail(t2)[sector].min() == 0) or (amount_matrix[sector].sum() == 0):
            predictions[sector] = 0
            continue
        
        recent_vals = amount_matrix.tail(n_lags)[sector].values
        
        if len(recent_vals) == n_lags and (recent_vals > 0).any():
            positive_mask = recent_vals > 0
            positive_vals = recent_vals[positive_mask]
            corresponding_weights = weights[positive_mask]
            
            if len(positive_vals) > 0:
                corresponding_weights = corresponding_weights / corresponding_weights.sum()
                log_vals = np.log(positive_vals)
                weighted_log_mean = np.sum(corresponding_weights * log_vals)
                weighted_geom_mean = np.exp(weighted_log_mean)
                predictions[sector] = weighted_geom_mean
            else:
                predictions[sector] = 0
        else:
            predictions[sector] = 0
    
    return predictions

def compute_december_multipliers(amount_matrix, clip_low=0.85, clip_high=1.40):
    """Compute December seasonality multipliers"""
    is_december = (amount_matrix.index.values % 12) == 11
    dec_means = amount_matrix[is_december].mean(axis=0)
    nondec_means = amount_matrix[~is_december].mean(axis=0)
    dec_counts = amount_matrix[is_december].notna().sum(axis=0)
    
    raw_mult = dec_means / (nondec_means + 1e-9)
    overall_mult = float(dec_means.mean() / (nondec_means.mean() + 1e-9))
    
    raw_mult = raw_mult.where(dec_counts >= 1, overall_mult)
    raw_mult = raw_mult.replace([np.inf, -np.inf], 1.0).fillna(1.0)
    clipped_mult = raw_mult.clip(lower=clip_low, upper=clip_high)
    
    return clipped_mult.to_dict()

def ewgm_per_sector(amount_matrix, sector, n_lags, alpha):
    """Exponential weighted geometric mean for one sector"""
    weights = np.array([alpha**(n_lags - 1 - i) for i in range(n_lags)], dtype=float)
    weights = weights / weights.sum()
    
    recent_vals = amount_matrix.tail(n_lags)[sector].values
    if (len(recent_vals) != n_lags) or (recent_vals <= 0).all():
        return 0.0
    
    mask = recent_vals > 0
    pos_vals = recent_vals[mask]
    pos_w = weights[mask]
    
    if pos_vals.size == 0:
        return 0.0
    
    pos_w = pos_w / pos_w.sum()
    log_vals = np.log(pos_vals + 1e-12)
    wlm = np.sum(pos_w * log_vals)
    return float(np.exp(wlm))

def seasonal_bump_prediction(amount_matrix, n_lags=7, alpha=0.5, t2=6, 
                            clip_low=0.85, clip_high=1.40, prediction_horizon=12):
    """Method 2: Seasonality Bump"""
    # Base predictions
    last_time = amount_matrix.index.max()
    pred_indices = np.arange(last_time + 1, last_time + 1 + prediction_horizon)
    predictions = pd.DataFrame(index=pred_indices, columns=amount_matrix.columns, dtype=float)
    
    for sector in amount_matrix.columns:
        if (amount_matrix.tail(t2)[sector].min() == 0) or (amount_matrix[sector].sum() == 0):
            predictions[sector] = 0.0
            continue
        
        base = ewgm_per_sector(amount_matrix, sector, n_lags, alpha)
        predictions[sector] = base
    
    # Apply December multipliers
    dec_multipliers = compute_december_multipliers(amount_matrix, clip_low, clip_high)
    
    for time_idx in predictions.index:
        if (time_idx % 12) == 11:  # December
            for sector in predictions.columns:
                m = dec_multipliers.get(sector, 1.0)
                predictions.loc[time_idx, sector] *= m
    
    return predictions


 ## 4. Stacking Meta-Learning Functions

In [52]:
def calculate_custom_metric(y_pred, y_true):
    """
    Implements the two-stage custom metric from competition
    
    Stage 1: If >30% of samples have APE > 100%, return score of 0
    Stage 2: Calculate MAPE on samples with APE <= 100%, 
             divide by fraction of valid samples, subtract from 1
    """
    y_pred = np.array(y_pred).flatten()
    y_true = np.array(y_true).flatten()
    
    # Calculate absolute percentage errors
    ape = np.abs((y_pred - y_true) / (y_true + 1e-9))
    
    # Stage 1: Check if >30% have APE > 100%
    high_error_fraction = (ape > 1.0).sum() / len(ape)
    
    if high_error_fraction > 0.3:
        return 0.0
    
    # Stage 2: Calculate on samples with APE <= 100%
    valid_mask = ape <= 1.0
    
    if valid_mask.sum() == 0:
        return 0.0
    
    valid_ape = ape[valid_mask]
    mape = valid_ape.mean()
    
    # Scale by fraction of valid samples
    fraction_valid = valid_mask.sum() / len(ape)
    scaled_mape = mape / fraction_valid
    
    # Final score
    score = 1 - scaled_mape
    
    return score

def create_meta_features(amount_matrix, method1_preds, method2_preds, 
                        time_indices, lookback_windows=[3, 6, 12],
                        include_month=True):
    """
    Create meta-features for stacking
    
    Features:
    1. Method 1 predictions
    2. Method 2 predictions
    3. Historical averages (3, 6, 12 months)
    4. Month number (optional)
    """
    features_list = []
    targets_list = []
    time_list = []
    sector_list = []
    
    for time_idx in time_indices:
        for sector in amount_matrix.columns:
            # Base model predictions
            pred1 = method1_preds.loc[time_idx, sector]
            pred2 = method2_preds.loc[time_idx, sector]
            
            # Historical averages
            hist_features = []
            for window in lookback_windows:
                if time_idx >= window:
                    hist_avg = amount_matrix.iloc[time_idx-window:time_idx][sector].mean()
                else:
                    hist_avg = amount_matrix.iloc[:time_idx][sector].mean() if time_idx > 0 else 0
                hist_features.append(hist_avg)
            
            # Month feature
            month_num = time_idx % 12
            
            # Combine features
            feature_row = [pred1, pred2] + hist_features
            if include_month:
                feature_row.append(month_num)
            
            features_list.append(feature_row)
            time_list.append(time_idx)
            sector_list.append(sector)
            
            # Target (if available)
            if time_idx in amount_matrix.index:
                targets_list.append(amount_matrix.loc[time_idx, sector])
            else:
                targets_list.append(np.nan)
    
    # Create feature names
    feature_names = ['method1_pred', 'method2_pred'] + \
                   [f'hist_avg_{w}m' for w in lookback_windows]
    if include_month:
        feature_names.append('month_num')
    
    X = pd.DataFrame(features_list, columns=feature_names)
    X['time'] = time_list
    X['sector'] = sector_list
    y = pd.Series(targets_list, name='target')
    
    return X, y

def train_meta_model(X_meta, y_meta, model_type='ridge', alpha=1.0):
    """
    Train meta-model on validation data
    
    model_type: 'ridge', 'lasso', or 'rf'
    """
    # Remove rows with missing targets
    valid_mask = ~y_meta.isna()
    X_train = X_meta[valid_mask].drop(['time', 'sector'], axis=1)
    y_train = y_meta[valid_mask]
    
    # Select meta-model
    if model_type == 'ridge':
        meta_model = Ridge(alpha=alpha)
    elif model_type == 'lasso':
        meta_model = Lasso(alpha=alpha)
    elif model_type == 'rf':
        meta_model = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    else:
        raise ValueError(f"Unknown model type: {model_type}")
    
    print(f"\nTraining {model_type} meta-model on {len(X_train)} samples...")
    meta_model.fit(X_train, y_train)
    print("✓ Meta-model trained!")
    
    # Show feature importance
    if hasattr(meta_model, 'coef_'):
        print("\nLearned coefficients:")
        for name, coef in zip(X_train.columns, meta_model.coef_):
            print(f"  {name:20s}: {coef:8.4f}")
        print(f"  {'intercept':20s}: {meta_model.intercept_:8.4f}")
    
    return meta_model

def predict_with_meta_model(meta_model, X_meta):
    """Generate predictions using trained meta-model"""
    X_pred = X_meta.drop(['time', 'sector'], axis=1)
    
    # Fill NaN values with 0 (important for test set where some historical windows may be incomplete)
    X_pred = X_pred.fillna(0)
    
    predictions = meta_model.predict(X_pred)
    
    # Reshape to time x sector format
    result = pd.DataFrame({
        'time': X_meta['time'],
        'sector': X_meta['sector'],
        'prediction': predictions
    })
    
    pivot = result.pivot(index='time', columns='sector', values='prediction')
    return pivot


 ## 5. Train Meta-Model on Validation Data

In [53]:
print("=" * 60)
print("STACKING ENSEMBLE - TRAINING PHASE")
print("=" * 60)

# Split data for meta-model training
val_months = CONFIG_STACKING['validation_months']
train_cutoff = len(amount_matrix) - val_months

train_matrix = amount_matrix.iloc[:train_cutoff].copy()
val_matrix = amount_matrix.iloc[train_cutoff:].copy()
val_indices = val_matrix.index

print(f"\nData split:")
print(f"  Training: months {train_matrix.index.min()} to {train_matrix.index.max()}")
print(f"  Validation: months {val_indices.min()} to {val_indices.max()}")


STACKING ENSEMBLE - TRAINING PHASE

Data split:
  Training: months 0 to 60
  Validation: months 61 to 66


In [54]:
# Generate base model predictions on validation period
print("\nGenerating base model predictions on validation set...")
method1_val = weighted_geometric_mean_prediction(
    train_matrix, 
    **CONFIG_METHOD1,
    prediction_horizon=val_months
)
method2_val = seasonal_bump_prediction(
    train_matrix,
    **CONFIG_METHOD2,
    prediction_horizon=val_months
)

# Pre-scale predictions for regime shift
pre_scale = CONFIG_STACKING.get('pre_scale_factor', 1.0)
if pre_scale != 1.0:
    print(f"\nPre-scaling predictions by {pre_scale} (regime shift adjustment)")
    method1_val = method1_val * pre_scale
    method2_val = method2_val * pre_scale

print(f"  Method 1 predictions: {method1_val.shape}")
print(f"  Method 2 predictions: {method2_val.shape}")



Generating base model predictions on validation set...

Pre-scaling predictions by 0.85 (regime shift adjustment)
  Method 1 predictions: (6, 96)
  Method 2 predictions: (6, 96)


In [55]:
# Create meta-features for validation
X_meta_val, y_meta_val = create_meta_features(
    amount_matrix,
    method1_val,
    method2_val,
    val_indices,
    lookback_windows=CONFIG_STACKING['lookback_windows'],
    include_month=CONFIG_STACKING['include_month_feature']
)

print(f"\nMeta-features shape: {X_meta_val.shape}")
print(f"Features: {[c for c in X_meta_val.columns if c not in ['time', 'sector']]}")



Meta-features shape: (576, 8)
Features: ['method1_pred', 'method2_pred', 'hist_avg_3m', 'hist_avg_6m', 'hist_avg_12m', 'month_num']


In [56]:
# Train meta-model
meta_model = train_meta_model(
    X_meta_val,
    y_meta_val,
    model_type=CONFIG_STACKING['meta_model'],
    alpha=CONFIG_STACKING['meta_alpha']
)



Training ridge meta-model on 576 samples...
✓ Meta-model trained!

Learned coefficients:
  method1_pred        :  -1.8436
  method2_pred        :   2.6156
  hist_avg_3m         :   0.2784
  hist_avg_6m         :   0.4781
  hist_avg_12m        :  -0.2259
  month_num           : 2738.5963
  intercept           : -9576.6339


 ## 6. Generate Final Predictions on Test Set

In [57]:
print("\n" + "=" * 60)
print("GENERATING FINAL PREDICTIONS")
print("=" * 60)

# Generate base model predictions on full training data for test period
print("\nGenerating base model predictions on test set...")
method1_test = weighted_geometric_mean_prediction(
    amount_matrix,
    **CONFIG_METHOD1,
    prediction_horizon=12
)
method2_test = seasonal_bump_prediction(
    amount_matrix,
    **CONFIG_METHOD2,
    prediction_horizon=12
)

# Pre-scale predictions for regime shift
pre_scale = CONFIG_STACKING.get('pre_scale_factor', 1.0)
if pre_scale != 1.0:
    print(f"\nPre-scaling test predictions by {pre_scale} (regime shift adjustment)")
    method1_test = method1_test * pre_scale
    method2_test = method2_test * pre_scale

test_indices = method1_test.index



GENERATING FINAL PREDICTIONS

Generating base model predictions on test set...

Pre-scaling test predictions by 0.85 (regime shift adjustment)


In [58]:
# Create meta-features for test
X_meta_test, _ = create_meta_features(
    amount_matrix,
    method1_test,
    method2_test,
    test_indices,
    lookback_windows=CONFIG_STACKING['lookback_windows'],
    include_month=CONFIG_STACKING['include_month_feature']
)

# Generate final predictions
final_predictions = predict_with_meta_model(meta_model, X_meta_test)

# Clip negative predictions to zero (transaction amounts can't be negative)
final_predictions = final_predictions.clip(lower=0)

# Ensure sector 95 is all zeros (no training data for this sector)
if 95 in final_predictions.columns:
    final_predictions[95] = 0

print(f"\nFinal predictions shape: {final_predictions.shape}")
print(f"Prediction statistics:")
print(f"  Min: {final_predictions.min().min():,.0f}")
print(f"  Max: {final_predictions.max().max():,.0f}")
print(f"  Mean: {final_predictions.mean().mean():,.0f}")
print(f"  Median: {final_predictions.median().median():,.0f}")
print(f"  Sector 95 predictions (should be all 0): {final_predictions[95].unique()}")



Final predictions shape: (12, 96)
Prediction statistics:
  Min: 0
  Max: 264,031
  Mean: 25,692
  Median: 10,278
  Sector 95 predictions (should be all 0): [0]


 ## 7. Create Submission

In [59]:
print("\n" + "=" * 60)
print("CREATING SUBMISSION")
print("=" * 60)

submission = test.copy()

# Map predictions to test set
prediction_values = []
for _, row in test.iterrows():
    time_idx = row['time']
    sector_id = row['sector_id']
    pred_value = final_predictions.loc[time_idx, sector_id]
    prediction_values.append(pred_value)

submission['new_house_transaction_amount'] = prediction_values

# Create output directory if it doesn't exist
output_dir = Path(CONFIG_OUTPUT['output_path'])
output_dir.mkdir(parents=True, exist_ok=True)

# Save submission
output_file = output_dir / CONFIG_OUTPUT['filename']
submission[['id', 'new_house_transaction_amount']].to_csv(output_file, index=False)

print(f"\n✅ Stacking submission created successfully!")
print(f"Saved to: {output_file}")
print(f"\nFirst few predictions:")
print(submission[['id', 'new_house_transaction_amount']].head(10))



CREATING SUBMISSION

✅ Stacking submission created successfully!
Saved to: /Users/nikola/Python/KaggleCompetition/output/16_stacking_ensemble/stacking_ensemble_submission.csv

First few predictions:
                   id  new_house_transaction_amount
0   2024 Aug_sector 1                  20080.278400
1   2024 Aug_sector 2                  15981.597490
2   2024 Aug_sector 3                  15696.319314
3   2024 Aug_sector 4                 103557.026326
4   2024 Aug_sector 5                  12381.231355
5   2024 Aug_sector 6                  27974.548520
6   2024 Aug_sector 7                  17947.385391
7   2024 Aug_sector 8                  13920.146785
8   2024 Aug_sector 9                  23577.488443
9  2024 Aug_sector 10                  70683.604152


 ## 8. Comparison with Fixed Weights

In [60]:
print("\n" + "=" * 60)
print("COMPARISON: Stacking vs Fixed Weights (on validation)")
print("=" * 60)

# Fixed weights approach (original best: 0.2, 0.65)
# Note: Apply same pre-scaling for fair comparison
pre_scale = CONFIG_STACKING.get('pre_scale_factor', 1.0)
method1_val_comparison = method1_val.copy()
method2_val_comparison = method2_val.copy()
fixed_weights_val = 0.2 * method1_val_comparison + 0.65 * method2_val_comparison

# Stacking approach on validation
stacking_val = predict_with_meta_model(meta_model, X_meta_val)

# Clip negative predictions
stacking_val = stacking_val.clip(lower=0)

print(f"\nNote: Both methods pre-scaled by {pre_scale} for regime shift")

# Compare predictions
comparison_df = pd.DataFrame({
    'Method1': method1_val_comparison.values.flatten(),
    'Method2': method2_val_comparison.values.flatten(),
    'Fixed_Weights': fixed_weights_val.values.flatten(),
    'Stacking': stacking_val.values.flatten(),
    'Actual': val_matrix.values.flatten()
})

print("\nValidation predictions summary:")
print(comparison_df.describe())

print("\nCorrelations with Actual:")
print(comparison_df.corr()['Actual'].sort_values(ascending=False))

# Calculate custom metric for each method
print("\n" + "=" * 60)
print("CUSTOM COMPETITION METRIC (higher is better)")
print("=" * 60)

score_method1 = calculate_custom_metric(
    comparison_df['Method1'].values,
    comparison_df['Actual'].values
)
score_method2 = calculate_custom_metric(
    comparison_df['Method2'].values,
    comparison_df['Actual'].values
)
score_fixed = calculate_custom_metric(
    comparison_df['Fixed_Weights'].values,
    comparison_df['Actual'].values
)
score_stacking = calculate_custom_metric(
    comparison_df['Stacking'].values,
    comparison_df['Actual'].values
)

print(f"\nMethod 1 (WGM):          {score_method1:.5f}")
print(f"Method 2 (Seasonal):     {score_method2:.5f}")
print(f"Fixed Weights (0.2/0.65): {score_fixed:.5f}")
print(f"Stacking (pre-scaled):    {score_stacking:.5f}")

# Show improvement
if score_stacking > score_fixed:
    improvement = score_stacking - score_fixed
    pct_improvement = (improvement / score_fixed) * 100
    print(f"\n✅ Stacking improvement: +{improvement:.5f} ({pct_improvement:+.2f}%)")
elif score_stacking < score_fixed:
    decline = score_fixed - score_stacking
    pct_decline = (decline / score_fixed) * 100
    print(f"\n❌ Stacking decline: -{decline:.5f} ({pct_decline:.2f}%)")
else:
    print(f"\n➡️ No difference between methods")

print("\n" + "=" * 60)
print("DONE!")
print("=" * 60)



COMPARISON: Stacking vs Fixed Weights (on validation)

Note: Both methods pre-scaled by 0.85 for regime shift

Validation predictions summary:
            Method1       Method2  Fixed_Weights       Stacking         Actual
count    576.000000    576.000000     576.000000     576.000000     576.000000
mean   16353.109759  16348.670186   13897.257573   23983.653512   23522.521667
std    19336.026877  19326.290151   16429.270003   28088.415915   35056.964536
min        0.000000      0.000000       0.000000       0.000000       0.000000
25%     1696.429766   1688.843128    1437.033987    4116.347365    2286.047500
50%    10637.179542  10623.991032    9026.243598   14493.203377   10565.440000
75%    25029.982222  25028.959368   21274.820034   33182.081690   27081.057500
max    92087.268502  92062.672941   78258.191112  137109.160304  272474.460000

Correlations with Actual:
Actual           1.000000
Stacking         0.815521
Method1          0.785926
Fixed_Weights    0.785866
Method2       

 ## Summary



 This stacking ensemble:

 - Trains a Ridge model to learn optimal combinations of base predictions

 - Uses historical averages (3m, 6m, 12m) as additional features

 - Learns sector-specific and month-specific weights automatically



 Expected improvement over fixed weights (0.2, 0.65): +0.0005 to +0.003

In [61]:
# %% [code]
# Compare stacking submission with best fixed weights submission
print("=" * 60)
print("CORRELATION CHECK: Stacking vs Best Fixed Weights")
print("=" * 60)

# Load both submission files
stacking_path = '/Users/nikola/Python/KaggleCompetition/output/16_stacking_ensemble/stacking_ensemble_submission.csv'
ensemble_path = '/Users/nikola/Python/KaggleCompetition/output/15_new_try_EWGM_Ensemble/15_EWGM_w85_submission.csv'

stacking_sub = pd.read_csv(stacking_path)
ensemble_sub = pd.read_csv(ensemble_path)

# Merge on id to align predictions
merged = stacking_sub.merge(ensemble_sub, on='id', suffixes=('_stacking', '_ensemble'))

# Calculate correlation
correlation = merged['new_house_transaction_amount_stacking'].corr(
    merged['new_house_transaction_amount_ensemble']
)

print(f"\nCorrelation between submissions: {correlation:.5f}")

# Show prediction statistics
print(f"\nPrediction statistics:")
print(merged[['new_house_transaction_amount_stacking', 'new_house_transaction_amount_ensemble']].describe())

# Check differences
merged['abs_diff'] = np.abs(
    merged['new_house_transaction_amount_stacking'] - 
    merged['new_house_transaction_amount_ensemble']
)
merged['pct_diff'] = merged['abs_diff'] / (merged['new_house_transaction_amount_ensemble'] + 1)

print(f"\nDifferences:")
print(f"  Mean absolute difference: {merged['abs_diff'].mean():,.2f}")
print(f"  Max absolute difference: {merged['abs_diff'].max():,.2f}")
print(f"  Samples with >10% difference: {(merged['pct_diff'] > 0.1).sum()} / {len(merged)} ({(merged['pct_diff'] > 0.1).mean()*100:.1f}%)")
print(f"  Samples with >50% difference: {(merged['pct_diff'] > 0.5).sum()} / {len(merged)} ({(merged['pct_diff'] > 0.5).mean()*100:.1f}%)")

if correlation < 0.90:
    print(f"\nWarning: Low correlation ({correlation:.3f}) - Stacking diverging significantly")
    print("This likely explains the 0.0 Kaggle score")
elif correlation > 0.99:
    print(f"\nHigh correlation ({correlation:.3f}) - Stacking barely different from fixed weights")
    print("Stacking isn't adding value")
else:
    print(f"\nModerate correlation ({correlation:.3f}) - Stacking making adjustments")

# Show worst divergences
print(f"\nTop 10 largest differences:")
print(merged.nlargest(10, 'abs_diff')[['id', 'new_house_transaction_amount_ensemble', 
                                        'new_house_transaction_amount_stacking', 'abs_diff']])

CORRELATION CHECK: Stacking vs Best Fixed Weights

Correlation between submissions: 0.76627

Prediction statistics:
       new_house_transaction_amount_stacking  \
count                            1152.000000   
mean                            25691.861979   
std                             35006.031171   
min                                 0.000000   
25%                              4104.681848   
50%                             15057.236695   
75%                             31720.041084   
max                            264031.303661   

       new_house_transaction_amount_ensemble  
count                            1152.000000  
mean                            21061.740699  
std                             28002.463975  
min                                 0.000000  
25%                              1843.147180  
50%                              9853.129411  
75%                             25620.303156  
max                            145102.206947  

Differences:
  Mean absolut